In [1]:
# Libraries
import time
import pandas as pd  

# ------------- # 

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# library for interacting with the operating system
import os

# library for directory location:

import pathlib
import os
from os.path import join

#Ignore warning -- Some methods are going to be deprecated and I didn't change all (mainly in the function scrapper)
import warnings
warnings.filterwarnings('ignore')

import multiprocessing

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Maximize Window
driver.maximize_window() 
driver.minimize_window()  
driver.maximize_window()  
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

# Enter to the site
driver.get('https://www.linkedin.com/login');
time.sleep(2)

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:01<00:00, 4.86MB/s]


In [3]:
# Accept cookies
#driver.find_element(By.XPATH, "/html/body/div/main/div[1]/div/section/div/div[2]/button[2]").click()

# User Credentials
user_name = 'quino117@hotmail.com'
password = 'Lancelot71.'
driver.find_element(By.XPATH,'//*[@id="username"]').send_keys(user_name)
driver.find_element(By.XPATH,'//*[@id="password"]').send_keys(password)
time.sleep(1)

# Login button
driver.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button').click()
driver.implicitly_wait(30)


In [4]:
# Jobs page
driver.find_element(By.XPATH,'//*[@id="global-nav"]/div/nav/ul/li[3]/a').click()
time.sleep(3)

# Go to search results directly
driver.get("https://www.linkedin.com/jobs/search/?geoId=105646813&keywords=data%20analyst&location=Spain")
time.sleep(1)

In [5]:
# Get all links for these offers
links = []

# Navigate 20 pages
print('Links are being collected now.')

try: 
    for page in range(2,15):
        time.sleep(2)
        jobs_block = driver.find_element(By.CLASS_NAME,'jobs-search-results-list')
        jobs_list= jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')
    
        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME,'a')
            for a in all_links:
                if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in links: 
                    links.append(a.get_attribute('href'))
                else:
                    pass
            # scroll down for each job element
            driver.execute_script("arguments[0].scrollIntoView();", job)
        
        print(f'Collecting the links in the page: {page-1}')
        # go to next page:
        driver.find_element(By.XPATH, f"//button[@aria-label='Page {page}']").click()
        time.sleep(2)
except:
    pass
print('Found ' + str(len(links)) + ' links for job offers')

Links are being collected now.
Found 324 links for job offers


In [6]:
# Create empty lists to store information
job_titles = []
company_names = []
company_locations = []
work_methods = []
post_dates = []
work_times = [] 
job_desc = []

i = 0
j = 1
# Visit each link one by one to scrape the information
print('Visiting the links and collecting information just started.')
for i in range(len(links)):
    try:
        driver.get(links[i])
        i=i+1
        time.sleep(2)
        # Click See more.
        driver.find_element(By.CLASS_NAME,"artdeco-card__actions").click()
        time.sleep(2)
    except:
        pass
    
    # Find the general information of the job offers
    contents = driver.find_elements(By.CLASS_NAME,'p5')
    for content in contents:
        try:
            job_titles.append(content.find_element(By.TAG_NAME,"h1").text)
            company_names.append(content.find_element(By.CLASS_NAME,"jobs-unified-top-card__company-name").text)
            company_locations.append(content.find_element(By.CLASS_NAME,"jobs-unified-top-card__bullet").text)
            work_methods.append(content.find_element(By.CLASS_NAME,"jobs-unified-top-card__workplace-type").text)
            post_dates.append(content.find_element(By.CLASS_NAME,"jobs-unified-top-card__posted-date").text)
            work_times.append(content.find_element(By.CLASS_NAME,"jobs-unified-top-card__job-insight").text)
            print(f'Scraping the Job Offer {j} DONE.')
            j+= 1
            
        except:
            pass
        time.sleep(1.5)
        
        # Scraping the job description
    job_description = driver.find_elements(By.CLASS_NAME,'jobs-description__content')
    for description in job_description:
        job_text = description.find_element(By.CLASS_NAME,"jobs-box__html-content").text
        job_desc.append(job_text)
        print(f'Scraping the Job Offer {j}')
        time.sleep(1.75)  
            
# Creating the dataframe 
df = pd.DataFrame(list(zip(job_titles,company_names,
                    company_locations,work_methods,
                    post_dates,work_times)),
                    columns =['job_title', 'company_name',
                           'company_location','work_method',
                           'post_date','work_time'])


Visiting the links and collecting information just started.
Scraping the Job Offer 1 DONE.
Scraping the Job Offer 2
Scraping the Job Offer 2 DONE.
Scraping the Job Offer 3
Scraping the Job Offer 3 DONE.
Scraping the Job Offer 4
Scraping the Job Offer 4 DONE.
Scraping the Job Offer 5
Scraping the Job Offer 5 DONE.
Scraping the Job Offer 6
Scraping the Job Offer 6 DONE.
Scraping the Job Offer 7
Scraping the Job Offer 7 DONE.
Scraping the Job Offer 8
Scraping the Job Offer 8 DONE.
Scraping the Job Offer 9
Scraping the Job Offer 9 DONE.
Scraping the Job Offer 10
Scraping the Job Offer 10 DONE.
Scraping the Job Offer 11
Scraping the Job Offer 11 DONE.
Scraping the Job Offer 12
Scraping the Job Offer 12 DONE.
Scraping the Job Offer 13
Scraping the Job Offer 13 DONE.
Scraping the Job Offer 14
Scraping the Job Offer 14 DONE.
Scraping the Job Offer 15
Scraping the Job Offer 15 DONE.
Scraping the Job Offer 16
Scraping the Job Offer 16 DONE.
Scraping the Job Offer 17
Scraping the Job Offer 17 DON

Scraping the Job Offer 137 DONE.
Scraping the Job Offer 138
Scraping the Job Offer 138 DONE.
Scraping the Job Offer 139
Scraping the Job Offer 139 DONE.
Scraping the Job Offer 140
Scraping the Job Offer 140 DONE.
Scraping the Job Offer 141
Scraping the Job Offer 141 DONE.
Scraping the Job Offer 142
Scraping the Job Offer 142 DONE.
Scraping the Job Offer 143
Scraping the Job Offer 143 DONE.
Scraping the Job Offer 144
Scraping the Job Offer 144
Scraping the Job Offer 144 DONE.
Scraping the Job Offer 145
Scraping the Job Offer 145 DONE.
Scraping the Job Offer 146
Scraping the Job Offer 146 DONE.
Scraping the Job Offer 147
Scraping the Job Offer 147 DONE.
Scraping the Job Offer 148
Scraping the Job Offer 148 DONE.
Scraping the Job Offer 149
Scraping the Job Offer 149 DONE.
Scraping the Job Offer 150
Scraping the Job Offer 150 DONE.
Scraping the Job Offer 151
Scraping the Job Offer 151 DONE.
Scraping the Job Offer 152
Scraping the Job Offer 152 DONE.
Scraping the Job Offer 153
Scraping the 

Scraping the Job Offer 254 DONE.
Scraping the Job Offer 255
Scraping the Job Offer 255 DONE.
Scraping the Job Offer 256
Scraping the Job Offer 256 DONE.
Scraping the Job Offer 257
Scraping the Job Offer 257
Scraping the Job Offer 257 DONE.
Scraping the Job Offer 258
Scraping the Job Offer 258 DONE.
Scraping the Job Offer 259
Scraping the Job Offer 259 DONE.
Scraping the Job Offer 260
Scraping the Job Offer 260 DONE.
Scraping the Job Offer 261
Scraping the Job Offer 261 DONE.
Scraping the Job Offer 262
Scraping the Job Offer 262 DONE.
Scraping the Job Offer 263
Scraping the Job Offer 263 DONE.
Scraping the Job Offer 264
Scraping the Job Offer 264 DONE.
Scraping the Job Offer 265
Scraping the Job Offer 265 DONE.
Scraping the Job Offer 266
Scraping the Job Offer 266 DONE.
Scraping the Job Offer 267
Scraping the Job Offer 267 DONE.
Scraping the Job Offer 268
Scraping the Job Offer 268 DONE.
Scraping the Job Offer 269
Scraping the Job Offer 269 DONE.
Scraping the Job Offer 270
Scraping the 

In [7]:
# Storing the data to csv file
df.to_csv('job_offers.csv', index=False)

# Output job descriptions to txt file
with open('job_descriptions.txt', 'w',encoding="utf-8") as f:
    for line in job_desc:
        f.write(line)
        f.write('\n')

In [8]:
df

,job_title,company_name,company_location,work_method,post_date,work_time
0,Business Intelligence Analyst (Qlik Sense),Fujitsu,Greater Madrid Metropolitan Area,Hybrid,2 weeks ago,Full-time · Associate
1,Analista de datos,TYPSA ESTADÍSTICA Y SERVICIOS (TEyS),"Madrid, Community of Madrid, Spain",Hybrid,1 week ago,Full-time · Associate
2,Power BI Developer,Redslim,"Madrid, Community of Madrid, Spain",Hybrid,2 weeks ago,Full-time · Mid-Senior level
3,Business Intelligence Consultant,Bosonit,Greater Madrid Metropolitan Area,Hybrid,1 week ago,Full-time · Mid-Senior level
4,Business Intelligence Analyst,Zolva Spain,Greater Madrid Metropolitan Area,Hybrid,23 hours ago,Full-time
...,...,...,...,...,...,...
266,Corporate Data Analyst con GCP (100% remoto),atSistemas,Spain,On-site,2 weeks ago,Full-time · Associate
267,Senior Digital Analyst,NTT DATA Europe & Latam,"Madrid, Community of Madrid, Spain",On-site,1 day ago,Full-time · Associate
268,Data Engineer (SQL Server & Azure),UST España & Latam,Spain,Remote,2 weeks ago,Full-time · Associate
269,Senior Business Analyst - ADI Market Share,Nestlé,"Barcelona, Catalonia, Spain",Hybrid,23 hours ago,Full-time · Mid-Senior level
